## Extracting all PDF URLs (1 level deep) from a single URL passed as parameter

In [1]:
from bs4 import BeautifulSoup
from pathlib import Path
import requests
import re

def get_base_url(url):
    # Extract the base url: protocol, subdomain, and domain
    base_re = r'^.+?[^\/:](?=[?\/]|$)'
    return re.findall(base_re, url)[0]

def get_pdf_urls(url):
    # Extract the base url: protocol, subdomain, and domain
    base_url = get_base_url(url)
    
    # Stop processing if the request wasn't successful
    r = requests.get(url)
    if r.status_code != 200:
        return set()
    
    # Extract all a-tags from the website HTML
    bs = BeautifulSoup(r.content)
    a_tags =  bs.findAll('a')
    
    # Loop through the a-tags saving all href attributes that end with .pdf
    hrefs = []
    for tag in a_tags:
        if 'href' in tag.attrs.keys() and tag.attrs['href'].endswith('.pdf'):
            hrefs.append(tag.attrs['href'])
    
    # Convert all hrefs to URLs by adding the base URL if required
    pdf_links = [href if href.startswith('http') else base_url + href for href in hrefs]
    return set(pdf_links)

def get_urls(url):
    # Extract the base url: protocol, subdomain, and domain
    base_url = get_base_url(url)
    
    # Stop processing if the request wasn't successful
    r = requests.get(url)
    if r.status_code != 200:
        return
    
    # Extract all a-tags from the website HTML
    bs = BeautifulSoup(r.content)
    a_tags =  bs.findAll('a')
    
    # Extract content of href attribute from the a-tags and convert them into proper URLs using the base url
    hrefs = {tag.attrs['href'] for tag in a_tags if 'href' in tag.attrs.keys()}
    hrefs = {href if href.startswith('http') else base_url + href for href in hrefs}
    
    # Return only URLs that don't end with .pdf and that start with the base URL
    return {href for href in hrefs if href.startswith(base_url) and not href.endswith('.pdf')}

def extract_pdf_urls_from_url(url, levels = 1):
    # Get all pdf URLs and other URLs
    pdf_urls = get_pdf_urls(url)
    other_urls = get_urls(url)
    
    # Loop through the other URLs extracting PDF URLs from each one of them
    if levels > 0:
        for i, url in enumerate(other_urls):
            print(f"{i+1}/{len(other_urls)} (#PDFs until now: {len(pdf_urls)}): {url}")
            pdfs_inside_url = get_pdf_urls(url)
            pdf_urls.update(pdfs_inside_url)
    
    # Return the list of all PDF URLs
    return pdf_urls

def download_pdfs(urls, save_path):
    # Create/Save path where to store all PDFs
    path = Path(save_path)
    path.mkdir(parents=True, exist_ok=True)
    
    # Loop through the set of PDF URLs, get its content and save the files
    for i, pdf_url in enumerate(urls):
        file = Path(str(i) + ".pdf")
        r = requests.get(pdf_url, stream=True)
        with open(path.joinpath(file), 'wb') as f:
            f.write(r.content)

url = 'https://www.cibc.com/en/about-cibc/investor-relations/quarterly-results.html'
pdf_urls = extract_pdf_urls_from_url(url, levels=0)
# download_pdfs(pdf_urls, 'data/test/path')